In [1]:
from rev_lib import *
import time
from matplotlib import pyplot as plt

In [2]:
gl = params(10,5,3,'c') #cynical model
#gl = params(10,9,3,'c') #cynical with 9 friends
#gl = params(10,5,3,'q') #quality model
#gl = params(10,9,3,'q') #quality with 9 friends

In [3]:
def sample_suggested(marg_pi,N=3):
    marg = 1.0*marg_pi
    l = []
    for i in range(N):
        #print(marg)
        #print(normalize(marg))
        sug_i = categorical(normalize(marg))
        marg[sug_i] = 0
        l.append(sug_i)
    return np.sort(l)

In [4]:
pi = normalize(gl.prior)
log_post = np.log(pi)
marg_pi = np.dot(pi,gl.configs)
c3 = third_more_likely(pi,gl)[0]

suggested_list = []
positives_list = []
margs_list =[]
c3_list=[]

In [5]:
def run_agressive_sim(gl):
    pi = normalize(gl.prior)
    log_post = np.log(pi)
    marg_pi = np.dot(pi,gl.configs)
    conf = third_more_likely(pi,gl)[0]

    suggested_list = []
    positives_list = []
    margs_list =[]
    conf_list=[]
    err_list=[]

    while(conf<.95):
        suggested = sample_suggested(marg_pi,N=3)

        if gl.model == 'c':
            positives = run_simulation_cynical([suggested],gl.ground)[0]
        elif gl.model == 'q':
            Q = gl.run_qualities(suggested.shape[0])
            positives = run_simulation_quality([suggested],Q,gl.ground)[0]

        li = point_log_like(suggested,positives,gl.model,gl.configs,gl.p_beta)
        log_post +=li
        log_post -=log_post.max()

        pi = normalize(np.exp(log_post))
        marg_pi = np.dot(pi,gl.configs)
        conf,prop_friends = third_more_likely(pi,gl)
        c3err = gl.N_suggested - np.sum(gl.ground[prop_friends])

        suggested_list.append(suggested)
        positives_list.append(positives)
        margs_list.append(marg_pi)
        conf_list.append(conf)
        err_list.append(c3err)

    return suggested_list,positives_list,conf_list,margs_list,err_list,len(conf_list),c3err

In [6]:
seq_suggested = []
seq_positives = []
seq_conf = []
seq_err = []

seq_nums=[]

for i in range(gl.S_slice[1]):
    suggested,positives,conf,marg_pi,errs,p3num,p3err = run_agressive_sim(gl)
    seq_suggested.append(suggested)
    seq_positives.append(positives)
    seq_conf.append(conf)
    seq_err.append(errs)
    seq_nums.append(p3num)
    #print(p3num,p3err)


In [7]:
suggested = np.vstack(seq_suggested)
cols = ['suggestion_{}'.format(i) for i in range(1,suggested[0].size+1)]
df = pd.DataFrame(suggested,columns=cols)

df['positives'] = np.hstack(seq_positives)
df['confidence'] = np.hstack(seq_conf)
df['errors'] = np.hstack(seq_err)

In [8]:
df2 = df[df['confidence'].to_numpy()>.95][['confidence','errors']]
df2['number'] = np.array(seq_nums)
df2 = df2[['number','confidence','errors']]

In [9]:
#Sanity check
np.all((df['confidence'].to_numpy()>.95)[np.cumsum(np.array(seq_nums))-1])

True

In [10]:
df.to_csv('sim_agressive_data_{}.csv'.format(gl.suffix),index=False)

In [11]:
df2.to_csv('agressive_prob3_errors_{}.csv'.format(gl.suffix),index=False)